In [1]:
import os
import time
import socket
import struct
import pickle
from tqdm import tqdm
import copy
import sys
import psutil
import random
random.seed(0)

import warnings
warnings.filterwarnings("ignore")

import torch
torch.manual_seed(0)
torch.initial_seed(), 
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn

device = 'cpu'
print(device)

cpu


In [2]:
BATCH_SIZE = 500

In [3]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 50

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [4]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = pickle.dumps(msg)
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    msg =  recvall(sock, msglen)
    msg = pickle.loads(msg)
    return msg

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

In [5]:
host = 'localhost'
port = 12313
s = socket.socket()
s.connect((host, port))
msg_r = recv_msg(s)

EPOCHS = msg_r['epochs']
USERS = msg_r['users']
CUT_LAYER = msg_r['cut_layer']
BOTTLENECK_COMPRESSION = msg_r['bottleneck_compression']
QUANTIZATION_REQUIRED = msg_r['quantization_required']
QUANTIZATION_TYPE = msg_r['QUANTIZATION_TYPE']

print(f"EPOCHS: {EPOCHS}")
print(f"USERS: {USERS}")
print(f"CUT_LAYER: {CUT_LAYER}")
print(f"BOTTLENECK_COMPRESSION: {BOTTLENECK_COMPRESSION}")
print(f"QUANTIZATION_REQUIRED: {QUANTIZATION_REQUIRED}")
print(f"QUANTIZATION_TYPE: {QUANTIZATION_TYPE}")

msg = len(trainloader)
send_msg(s, msg)   # send total_batch of train dataset

EPOCHS: 10
USERS: 1
CUT_LAYER: 1
BOTTLENECK_COMPRESSION: 4
QUANTIZATION_REQUIRED: 1
QUANTIZATION_TYPE: torch.qint8


In [6]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, self.expansion * out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [7]:
class ClientResNet(nn.Module):
    def __init__(self, block, output_channel=[64,128,256,512], stride=[1,2,2,2], num_blocks=[1,1,1,1], num_classes=10, 
                 bottleneck_compression=4, cut_layer=1):
        super(ClientResNet, self).__init__()
        
        self.in_channels = 64
        self.output_channel = output_channel
        self.stride = stride
        self.num_blocks = num_blocks
        self.num_classes = num_classes
        self.bottleneck_compression = bottleneck_compression
        self.cut_layer = cut_layer
        
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layers = []
        for layer in range(0, self.cut_layer):
            self.layers.append(self._make_layer(block, self.output_channel[layer], self.num_blocks[layer], stride=self.stride[layer]))
        self.conv_layers = nn.Sequential(*self.layers)

        if self.bottleneck_compression>=1:
            self.bl_encoder = nn.Conv2d(output_channel[self.cut_layer-1], output_channel[self.cut_layer-1]//bottleneck_compression, kernel_size=3, stride=1, padding=1, bias=False)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion
        return nn.Sequential(*layers)
                
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out)
        out = self.conv_layers(out)
        if self.bottleneck_compression>=1:
            out = self.bl_encoder(out)
        return out

# Create the model
client_model = ClientResNet(block=BasicBlock, bottleneck_compression=BOTTLENECK_COMPRESSION, cut_layer=CUT_LAYER)
client_model = client_model.to(device)
print(client_model)

ClientResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (conv_layers): Sequential(
    (0): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
    )
  )
  (bl_encoder): Conv2d(64, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)


In [8]:
def quantize_tensor(tensor, quantization_type):

    min_val, max_val = tensor.min(), tensor.max()

    if quantization_type == torch.quint8:
        scale = torch.tensor((max_val - min_val) / 255)
        zero_point = torch.tensor(max(0, min(255, int(-min_val / scale))))
        return torch.quantize_per_tensor(tensor, scale, zero_point, torch.quint8)

    elif quantization_type == torch.qint8:
        scale = torch.tensor(max_val / 127 if max_val.abs() > min_val.abs() else min_val / -128)
        zero_point = torch.tensor(0)  # For qint8, zero_point is typically set to 0 for symmetric quantization
        return torch.quantize_per_tensor(tensor, scale, zero_point, torch.qint8)

    else:
        raise ValueError("Invalid quantization type. Choose 'quint8' or 'qint8'.")

def dequantize_tensor(quantized_tensor):
    return torch.dequantize(quantized_tensor).requires_grad_(True)

In [9]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(client_model.parameters(), lr=0.001, momentum=0.9)

In [10]:
from tqdm import tqdm

for e in range(EPOCHS):
    client_weights = recv_msg(s)
    client_model.load_state_dict(client_weights, strict=False)
    client_model.eval()
    for i, data in enumerate(tqdm(trainloader, ncols=100, desc='Epoch '+str(e+1))):
        
        x, label = data
        
        x = x.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        
        output = client_model(x)

        if QUANTIZATION_REQUIRED==1:
            output_quantized = quantize_tensor(output, QUANTIZATION_TYPE)
            msg = {
                'client_output': output_quantized,
                'label': label,
            }

        else:
            msg = {
                'client_output': output,
                'label': label,
            }

        send_msg(s, msg)
        
        client_grad = recv_msg(s)
        
        if QUANTIZATION_REQUIRED==1:
            client_grad = dequantize_tensor(client_grad)

        output.backward(client_grad)
        optimizer.step()
    send_msg(s, client_model.state_dict())

Epoch 10: 100%|███████████████████████████████████████████████████| 100/100 [00:31<00:00,  3.19it/s]


In [11]:
torch.save(client_model.state_dict(), 'client_model_state_dict.pth')

# Replace 'path_to_file' with your file path
file_path = 'client_model_state_dict.pth'

# Get the size of the file in bytes
file_size = os.path.getsize(file_path)

print(f"The size of the file is {file_size / (1024**2)} MB.")


The size of the file is 0.3619508743286133 MB.


In [12]:
print(f"Current Memory Usage: {psutil.Process(os.getpid()).memory_info().rss / 1024 ** 2} MB")

Current Memory Usage: 910.2265625 MB
